## Brain Tumour Detection model using XGBoost Classifier and Hyperparameter Optimization using Raytune:

In [1]:
#pip install ray[tune]

In [2]:
# Importing Ray:
import ray
print('ray %s' % ray.__version__)

ray 1.11.0


In [3]:
# Importing required Libraries:
import os
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.xgboost import TuneReportCheckpointCallback

In [4]:
# Creating a Objective function:
def objective(config: dict):
    
    # Reading the dataset:
    df = pd.read_csv("/home/codebind/Project/HPO_models/Zernike_Moments_YN_3000.csv", header=None)
    
    # Shuffling the whole dataset:
    df = df.sample(frac=1, random_state=0).reset_index(drop=True)
    
    # Converting categorical values of Target feature into numerical:
    df[289].replace(['YES','NO'], [1,0], inplace=True)
    
    # Independent and Dependent features:
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    # Train-Test spilt:
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(x_train, label = y_train)
    test_set = xgb.DMatrix(x_test, label = y_test)
    
    # Train the classifier, using the Tune callback
    xgb.train(
              config,
              train_set,
              evals = [(test_set, "eval")],
              verbose_eval = False,
              callbacks = [TuneReportCheckpointCallback(filename="model.xgb")]
              )

def get_best_model_checkpoint(analysis):
    best_bst = xgb.Booster()
    best_bst.load_model(os.path.join(analysis.best_checkpoint, "model.xgb"))
    accuracy = 1. - analysis.best_result["eval-error"]
    
    print(f"Best model parameters: {analysis.best_config}")
    print(f"Best model total accuracy: {accuracy:.4f}")

    return best_bst

def tune_xgboost():

    search_space = {
                    "objective": "binary:logistic",
                    "learning_rate": tune.loguniform(0.0001, 2.0),
                    "eval_metric": ["logloss", "error"],
                    "max_depth": tune.randint(1, 20),
                    "min_child_weight": tune.choice([1,2,3,4,5,6,7,8,9,10])
                    }

    # This will enable aggressive early stopping of bad trials.

    scheduler = ASHAScheduler(
                              max_t = 100,  # 10 training iterations
                              grace_period = 1,
                              reduction_factor = 2
                             )


    analysis = tune.run(
                        objective,
                        metric = "eval-logloss",
                        mode = "min",
                        # You can add "gpu": 0.1 to allocate GPUs
                        resources_per_trial = {"cpu": 1},
                        config = search_space,
                        num_samples = 100,
                        scheduler = scheduler
                       )

    return analysis

if __name__ == "__main__":
    
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument(
                        "--server-address",
                        type=str,
                        default=None,
                        required=False,
                        help="The address of server to connect to if using Ray Client."
                        )
    
    args, _ = parser.parse_known_args()

    if args.server_address:
        
        import ray
        ray.init(f"ray://{args.server_address}")

    analysis = tune_xgboost()

    # Load the best model checkpoint.
    if args.server_address:
        # If connecting to a remote server with Ray Client, checkpoint loading
        # should be wrapped in a task so it will execute on the server.
        # We have to make sure it gets executed on the same node that
        # ``tune.run`` is called on.
        from ray.util.ml_utils.node import force_on_current_node
        remote_fn = force_on_current_node(ray.remote(get_best_model_checkpoint))
        best_bst = ray.get(remote_fn.remote(analysis))
    else:
        best_bst = get_best_model_checkpoint(analysis)

2022-04-08 15:16:59,224	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
/home/codebind/anaconda3/envs/my_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trial name,status,loc,learning_rate,max_depth,min_child_weight
objective_d59c9_00000,RUNNING,192.168.0.113:188506,0.00162463,7,6
objective_d59c9_00001,PENDING,,0.0205403,16,4
objective_d59c9_00002,PENDING,,0.158196,3,8
objective_d59c9_00003,PENDING,,0.00226189,13,5
objective_d59c9_00004,PENDING,,0.155815,2,10
objective_d59c9_00005,PENDING,,0.000864446,3,4
objective_d59c9_00006,PENDING,,0.18045,6,4
objective_d59c9_00007,PENDING,,0.0100158,15,6
objective_d59c9_00008,PENDING,,1.79891,2,5
objective_d59c9_00009,PENDING,,0.0861111,6,5


Result for objective_d59c9_00000:
  date: 2022-04-08_15-17-04
  done: false
  eval-error: 0.173913
  eval-logloss: 0.692197
  experiment_id: 6b056fcac3534e6ca90087e283d57bc5
  hostname: abhishek
  iterations_since_restore: 1
  node_ip: 192.168.0.113
  pid: 188506
  should_checkpoint: true
  time_since_restore: 2.5635578632354736
  time_this_iter_s: 2.5635578632354736
  time_total_s: 2.5635578632354736
  timestamp: 1649411224
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d59c9_00000
  


Trial name,status,loc,learning_rate,max_depth,min_child_weight,iter,total time (s),eval-logloss,eval-error
objective_d59c9_00000,RUNNING,192.168.0.113:188506,0.00162463,7,6,1,2.56356,0.692197,0.173913
objective_d59c9_00001,RUNNING,192.168.0.113:188503,0.0205403,16,4,,,,
objective_d59c9_00002,RUNNING,192.168.0.113:188505,0.158196,3,8,,,,
objective_d59c9_00003,RUNNING,192.168.0.113:188507,0.00226189,13,5,,,,
objective_d59c9_00004,RUNNING,192.168.0.113:188501,0.155815,2,10,,,,
objective_d59c9_00005,RUNNING,192.168.0.113:188508,0.000864446,3,4,,,,
objective_d59c9_00006,RUNNING,192.168.0.113:188502,0.18045,6,4,,,,
objective_d59c9_00007,RUNNING,192.168.0.113:188504,0.0100158,15,6,,,,
objective_d59c9_00008,PENDING,,1.79891,2,5,,,,
objective_d59c9_00009,PENDING,,0.0861111,6,5,,,,


Result for objective_d59c9_00005:
  date: 2022-04-08_15-17-09
  done: true
  eval-error: 0.248188
  eval-logloss: 0.692831
  experiment_id: cd2340056bf04e2ab1b2b4b025e54a61
  hostname: abhishek
  iterations_since_restore: 1
  node_ip: 192.168.0.113
  pid: 188508
  should_checkpoint: true
  time_since_restore: 2.973721504211426
  time_this_iter_s: 2.973721504211426
  time_total_s: 2.973721504211426
  timestamp: 1649411229
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d59c9_00005
  
Result for objective_d59c9_00002:
  date: 2022-04-08_15-17-09
  done: false
  eval-error: 0.248188
  eval-logloss: 0.640485
  experiment_id: 2f01338fcf8c41e0a7c224509c98962c
  hostname: abhishek
  iterations_since_restore: 1
  node_ip: 192.168.0.113
  pid: 188505
  should_checkpoint: true
  time_since_restore: 4.0282816886901855
  time_this_iter_s: 4.0282816886901855
  time_total_s: 4.0282816886901855
  timestamp: 1649411229
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d

...

Trial name,status,loc,learning_rate,max_depth,min_child_weight,iter,total time (s),eval-logloss,eval-error
objective_d59c9_00096,RUNNING,192.168.0.113:192530,0.63961,15,5,7,28.8217,0.195862,0.072464
objective_d59c9_00099,RUNNING,192.168.0.113:192585,0.947839,14,6,8,23.0547,0.228483,0.077899
objective_d59c9_00000,TERMINATED,192.168.0.113:188506,0.00162463,7,6,4,15.852,0.689287,0.137681
objective_d59c9_00001,TERMINATED,192.168.0.113:188503,0.0205403,16,4,2,14.5853,0.668036,0.126812
objective_d59c9_00002,TERMINATED,192.168.0.113:188505,0.158196,3,8,10,18.6578,0.443368,0.161232
objective_d59c9_00003,TERMINATED,192.168.0.113:188507,0.00226189,13,5,1,8.32958,0.69177,0.146739
objective_d59c9_00004,TERMINATED,192.168.0.113:188501,0.155815,2,10,10,15.2389,0.512347,0.201087
objective_d59c9_00005,TERMINATED,192.168.0.113:188508,0.000864446,3,4,1,2.97372,0.692831,0.248188
objective_d59c9_00006,TERMINATED,192.168.0.113:188502,0.18045,6,4,10,34.4516,0.26338,0.070652
objective_d59c9_00007,TERMINATED,192.168.0.113:188504,0.0100158,15,6,1,7.50261,0.687315,0.173913


Result for objective_d59c9_00099:
  date: 2022-04-08_15-22-52
  done: false
  eval-error: 0.07971
  eval-logloss: 0.220072
  experiment_id: 447714bf7bd841cfb3b75cc4a35dc311
  hostname: abhishek
  iterations_since_restore: 9
  node_ip: 192.168.0.113
  pid: 192585
  time_since_restore: 24.370765686035156
  time_this_iter_s: 1.3160429000854492
  time_total_s: 24.370765686035156
  timestamp: 1649411572
  timesteps_since_restore: 0
  training_iteration: 9
  trial_id: d59c9_00099
  
Result for objective_d59c9_00099:
  date: 2022-04-08_15-22-52
  done: true
  eval-error: 0.07971
  eval-logloss: 0.217451
  experiment_id: 447714bf7bd841cfb3b75cc4a35dc311
  experiment_tag: 99_learning_rate=0.94784,max_depth=14,min_child_weight=6
  hostname: abhishek
  iterations_since_restore: 10
  node_ip: 192.168.0.113
  pid: 192585
  time_since_restore: 25.26966118812561
  time_this_iter_s: 0.8988955020904541
  time_total_s: 25.26966118812561
  timestamp: 1649411572
  timesteps_since_restore: 0
  training_ite

Trial name,status,loc,learning_rate,max_depth,min_child_weight,iter,total time (s),eval-logloss,eval-error
objective_d59c9_00000,TERMINATED,192.168.0.113:188506,0.00162463,7,6,4,15.852,0.689287,0.137681
objective_d59c9_00001,TERMINATED,192.168.0.113:188503,0.0205403,16,4,2,14.5853,0.668036,0.126812
objective_d59c9_00002,TERMINATED,192.168.0.113:188505,0.158196,3,8,10,18.6578,0.443368,0.161232
objective_d59c9_00003,TERMINATED,192.168.0.113:188507,0.00226189,13,5,1,8.32958,0.69177,0.146739
objective_d59c9_00004,TERMINATED,192.168.0.113:188501,0.155815,2,10,10,15.2389,0.512347,0.201087
objective_d59c9_00005,TERMINATED,192.168.0.113:188508,0.000864446,3,4,1,2.97372,0.692831,0.248188
objective_d59c9_00006,TERMINATED,192.168.0.113:188502,0.18045,6,4,10,34.4516,0.26338,0.070652
objective_d59c9_00007,TERMINATED,192.168.0.113:188504,0.0100158,15,6,1,7.50261,0.687315,0.173913
objective_d59c9_00008,TERMINATED,192.168.0.113:188900,1.79891,2,5,2,2.31459,0.646065,0.237319
objective_d59c9_00009,TERMINATED,192.168.0.113:188910,0.0861111,6,5,10,42.0907,0.393507,0.085145


2022-04-08 15:22:53,488	INFO tune.py:639 -- Total run time: 354.27 seconds (353.13 seconds for the tuning loop).


Best model parameters: {'objective': 'binary:logistic', 'learning_rate': 0.7205291217445136, 'eval_metric': ['logloss', 'error'], 'max_depth': 8, 'min_child_weight': 1}
Best model total accuracy: 0.9601


In [5]:
# Reading the dataset:
df = pd.read_csv('/home/codebind/Project/HPO_models/Zernike_Moments_YN_3000.csv', header=None)

In [6]:
# Shuffling the whole dataset:
df = df.sample(frac=1, random_state=0).reset_index(drop=True)

In [7]:
# Converting categorical values of Target feature into numerical:
df[289].replace(['YES','NO'], [1,0], inplace=True)

In [8]:
# independent and dependent features:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [9]:
# Train-Test spilt
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
# Using Best parameters found from HyperParameter Optimization done using Optuna:
xgb = XGBClassifier(
                    learning_rate = 0.7205,
                    max_depth = 8,
                    min_child_weight = 1
                   )

In [13]:
# Fitting the Training data:
xgb.fit(x_train, y_train)

[15:27:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.7205,
              max_delta_step=0, max_depth=8, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
# Predicting the test data:
y_pred = xgb.predict(x_test)

### Results:

In [15]:
print("Accuracy :", round(accuracy_score(y_test, y_pred), 2)*100, "%")

Accuracy : 96.0 %


In [16]:
print("Precision :", round(precision_score(y_test, y_pred), 2)*100, "%")

Precision : 97.0 %


In [17]:
print("Recall :", round(recall_score(y_test, y_pred), 2)*100, "%")

Recall : 97.0 %


In [18]:
print("f1_Score :", round(f1_score(y_test, y_pred), 2)*100, "%")

f1_Score : 97.0 %


In [19]:
print("Confusion Matrix :")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix :
[[197  10]
 [ 10 335]]


In [20]:
print("Classification Report :")
print(classification_report(y_test, y_pred))

Classification Report :
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       207
           1       0.97      0.97      0.97       345

    accuracy                           0.96       552
   macro avg       0.96      0.96      0.96       552
weighted avg       0.96      0.96      0.96       552

